In [3]:
from functions import *

# Usage
folder_path = '/Input'  # Update with the correct folder path or leave as is if the script is in the same folder as the input folder
output_path = '/Output'  # Idem for the output folder

create_booklet(folder_path, output_path)